In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.utils import *

import re
from mistletoe import Document
from mistletoe.html_renderer import HtmlRenderer
from mistletoe.span_token import SpanToken
from mistletoe.block_token import BlockToken

In [ ]:
from IPython.display import HTML,Markdown,display
from mistletoe import markdown

In [ ]:
#| export
class Subscript(SpanToken):
    pattern = re.compile(r"~([^~\s]+)~")
    parse_inner = False
    def __init__(self, match): self.content = match.group(1)

class Superscript(SpanToken):
    pattern = re.compile(r"\^([^\^\s]+)\^")
    parse_inner = False
    def __init__(self, match): self.content = match.group(1)

class Highlight(SpanToken):
    pattern = re.compile(r"==([^=]+)==")
    parse_inner = True
    def __init__(self, match): pass

In [ ]:
#| export
emoji_map = {
    ':smile:':'😊', ':laughing:':'😆', ':joy:':'😂', ':grin:':'😁', ':wink:':'😉',
    ':heart:':'❤️', ':blue_heart:':'💙', ':green_heart:':'💚', ':yellow_heart:':'💛', ':purple_heart:':'💜',
    ':thumbsup:':'👍', ':thumbsdown:':'👎', ':clap:':'👏', ':wave:':'👋', ':raised_hands:':'🙌',
    ':rocket:':'🚀', ':fire:':'🔥', ':star:':'⭐', ':sparkles:':'✨', ':zap:':'⚡',
    ':check:':'✓', ':x:':'✗', ':warning:':'⚠️', ':no_entry:':'⛔', ':white_check_mark:':'✅',
    ':eyes:':'👀', ':thinking:':'🤔', ':tada:':'🎉', ':100:':'💯', ':muscle:':'💪',
    ':pray:':'🙏', ':ok_hand:':'👌', ':point_right:':'👉', ':point_left:':'👈',
    ':bug:':'🐛', ':construction:':'🚧', ':pencil:':'✏️', ':memo:':'📝', ':books:':'📚',
    ':bulb:':'💡', ':gear:':'⚙️', ':link:':'🔗', ':lock:':'🔒', ':key:':'🔑',
    ':package:':'📦', ':inbox_tray:':'📥', ':outbox_tray:':'📤', ':email:':'📧',
    ':camera:':'📷', ':computer:':'💻', ':iphone:':'📱', ':hourglass:':'⏳',
    ':sunny:':'☀️', ':cloud:':'☁️', ':umbrella:':'☂️', ':snowflake:':'❄️',
    ':pizza:':'🍕', ':coffee:':'☕', ':beer:':'🍺', ':cake:':'🍰',
    ':cat:':'🐱', ':dog:':'🐶', ':whale:':'🐳', ':snake:':'🐍'
}

class Emoji(SpanToken):
    pattern = re.compile(r':(\w+):')
    parse_inner = False
    def __init__(self, match): self.name = match.group(1)

In [ ]:
#| export
class FootnoteEntry(BlockToken):
    @staticmethod
    def start(line): return line.startswith('[^') and ']: ' in line
    
    @staticmethod
    def read(lines):
        line = next(lines)
        match = re.match(r'^\[\^(\w+)\]: (.+)$', line)
        return match
    
    def __init__(self, match):
        self.label = match.group(1)
        self.content = match.group(2)
        self.children = []

class FootnoteRef(SpanToken):
    pattern = re.compile(r'\[\^(\w+)\]')
    parse_inner = False
    def __init__(self, match): self.label = match.group(1)

In [ ]:
#| export
class Strikethrough(SpanToken):
    pattern = re.compile(r'~~([^~]+)~~')
    parse_inner = True
    def __init__(self, match): pass

class AutoLink(SpanToken):
    pattern = re.compile(r'(?<!\()(https?://[^\s<>]+)')
    parse_inner = False
    precedence = 6
    def __init__(self, match): self.target = match.group(1)

In [ ]:
#| export
class ExtendedHtmlRenderer(HtmlRenderer):
    def __init__(self, *args, **kwargs): 
        super().__init__(Subscript, Superscript, Highlight, Emoji, FootnoteRef, FootnoteEntry, Strikethrough, AutoLink, *args, **kwargs)
        self.footnotes = {}
    def render_subscript(self, token): return f'<sub>{token.content}</sub>'
    def render_superscript(self, token): return f'<sup>{token.content}</sup>'
    def render_highlight(self, token): return f'<mark>{self.render_inner(token)}</mark>'
    def render_strikethrough(self, token): return f'<del>{self.render_inner(token)}</del>'
    def render_emoji(self, token): return emoji_map.get(f':{token.name}:', f':{token.name}:')
    def render_auto_link(self, token): return f'<a href="{token.target}">{token.target}</a>'
    def render_footnote_ref(self, token): 
        return f'<sup><a href="#fn-{token.label}" id="fnref-{token.label}">[{token.label}]</a></sup>'
    def render_footnote_entry(self, token):
        return f'<div id="fn-{token.label}" class="footnote"><sup>{token.label}</sup> {token.content} <a href="#fnref-{token.label}">↩</a></div>\n'
    def render_list_item(self, token):
        inner = self.render_inner(token)
        match = re.match(r'\[( |x|X)\] ', inner)
        if not match: return f'<li>{inner}</li>\n'
        checked = 'checked' if match.group(1).lower() == 'x' else ''
        checkbox = f'<input type="checkbox" disabled {checked}>'
        content = inner[match.end():]
        return f'<li>{checkbox} {content}</li>\n'

In [ ]:
def render_md(md): return HTML(markdown(md, ExtendedHtmlRenderer))

In [ ]:
render_md("Here's a sentence with a footnote[^1] and another[^2].\n\n[^1]: First note.\n[^2]: Second note.")

In [ ]:
render_md("~~strikethrough~~ and **bold ~~strikethrough~~**")

In [ ]:
render_md("Check out https://fast.ai and http://example.com for more info!")

In [ ]:
render_md("Here's a sentence with a link to <http://www.example.org>.")

In [ ]:
render_md("""Here's some code:

```
https://fast.ai
~~strikethrough~~
:smile:
H~2~O
```

And outside the block: https://fast.ai ~~strikethrough~~ :smile: H~2~O""")


In [ ]:
render_md("H~2~O and E=mc^2^")

In [ ]:
test_md2 = """
- [x] Completed task
- [ ] Incomplete task
- Regular item"""
render_md(test_md2)

In [ ]:
render_md("==highlighted== and :smile: :rocket: :heart:")

In [ ]:
#| export
def parse_attrs(text):
    if not text: return ''
    pattern = r'#([^}\s]+)|\.([^}\s]+)|([\w-]+)="([^"]*)"'
    id_,classes,attrs = '',[],{}
    for match in re.finditer(pattern, text):
        id_m,cls,key,val = match.groups()
        if id_m: id_ = id_m
        elif cls: classes.append(cls)
        elif key: attrs[key] = val
    parts = []
    if id_: parts.append(f'id="{id_}"')
    if classes: parts.append(f'class="{" ".join(classes)}"')
    for k,v in attrs.items(): parts.append(f'{k}="{v}"')
    return ' ' + ' '.join(parts) if parts else ''

In [ ]:
parse_attrs('{#my-id .class1 .class2 width="50%" height="200 px"}')

' id="my-id" class="class1 class2" width="50%" height="200 px"'

In [ ]:
#| export
@patch
def render_heading(self:ExtendedHtmlRenderer, token):
    inner = self.render_inner(token)
    attr_match = re.search(r'\s*\{([^}]+)\}$', inner)
    if attr_match:
        attrs = parse_attrs(attr_match.group(0))
        inner = inner[:attr_match.start()]
        return f'<h{token.level}{attrs}>{inner}</h{token.level}>'
    return f'<h{token.level}>{inner}</h{token.level}>'

In [ ]:
%%html
<style>
.important { background-color: yellow; font-weight: bold; }
</style>

In [ ]:
render_md("# My Heading {#intro .important}")

## Tests

In [ ]:
from fastcore.test import test_eq

In [ ]:
with ExtendedHtmlRenderer() as renderer:
    test_eq(renderer.render(Document(':sm ile:')), '<p>:sm ile:</p>\n')
    test_eq(renderer.render(Document(':unknown:')), '<p>:unknown:</p>\n')
    test_eq(renderer.render(Document(':smile:')), '<p>😊</p>\n')
    test_eq(renderer.render(Document('H~ 2~O')), '<p>H~ 2~O</p>\n')
    test_eq(renderer.render(Document('H~2~O')), '<p>H<sub>2</sub>O</p>\n')
    test_eq(renderer.render(Document('E=mc^ 2^')), '<p>E=mc^ 2^</p>\n')
    test_eq(renderer.render(Document('E=mc^2^')), '<p>E=mc<sup>2</sup></p>\n')
    test_eq(renderer.render(Document('~~no space~~')), '<p><del>no space</del></p>\n')
    test_eq(renderer.render(Document('**~~nested~~**')), '<p><strong><del>nested</del></strong></p>\n')
    test_eq(renderer.render(Document('==~~double~~==')), '<p><mark><del>double</del></mark></p>\n')
    html = renderer.render(Document('```\nhttps://fast.ai\n```'))
    assert 'href' not in html and 'https://fast.ai' in html
    html = renderer.render(Document('https://fast.ai'))
    assert '<a href="https://fast.ai">https://fast.ai</a>' in html
    html = renderer.render(Document('[link](https://fast.ai)'))
    assert html.count('href') == 1
    test_eq(renderer.render(Document('[^1]')), '<p><sup><a href="#fn-1" id="fnref-1">[1]</a></sup></p>\n')
    html = renderer.render(Document('- [x] done\n- [ ] todo\n- regular'))
    assert 'checked' in html and html.count('checkbox') == 2

In [ ]:
with ExtendedHtmlRenderer() as renderer:
    test_eq(renderer.render(Document('# Heading {#intro}')), '<h1 id="intro">Heading</h1>\n')
    test_eq(renderer.render(Document('## Test {.important}')), '<h2 class="important">Test</h2>\n')
    test_eq(renderer.render(Document('### Multi {.c1 .c2}')), '<h3 class="c1 c2">Multi</h3>\n')
    html = renderer.render(Document('# Full {#id1 .cls data-level="1"}'))
    assert 'id="id1"' in html and 'class="cls"' in html and 'data-level="1"' in html
    test_eq(renderer.render(Document('# Plain')), '<h1>Plain</h1>\n')
    test_eq(renderer.render(Document('## Empty {}')), '<h2>Empty {}</h2>\n')
    html = renderer.render(Document('# Order {.first #myid .second key="val"}'))
    assert 'id="myid"' in html and 'class="first second"' in html and 'key="val"' in html
    test_eq(renderer.render(Document('# Spaces {data-value="hello world"}')), '<h1 data-value="hello world">Spaces</h1>\n')